# Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

# GPU 용량 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 최적 모델 찾기 (seed = 42)
- [최적 모델 찾기](https://snowdeer.github.io/machine-learning/2018/01/09/find-best-model/)
- 모델 생성 시 seed = 42 ~ 50일 때, 각각의 best모델에서 data seed = 1~50에서의 평균 정확도
- 모델 생성 시 seed = 42, 평균 정확도 : 82.6
- 모델 생성 시 seed = 43, 평균 정확도 : 81.6
- 모델 생성 시 seed = 44, 평균 정확도 : 82.5
- 모델 생성 시 seed = 45, 평균 정확도 : 82.5
- 모델 생성 시 seed = 46, 평균 정확도 : 81.6
- 모델 생성 시 seed = 47, 평균 정확도 : 81.8
- 모델 생성 시 seed = 48, 평균 정확도 : 81.1
- 모델 생성 시 seed = 49, 평균 정확도 : 82.3
- 모델 생성 시 seed = 50, 평균 정확도 : 80

## seed별 best model 저장

In [43]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 50
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 50 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed'f'{seed_num}''-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6754 - acc: 0.5949 - val_loss: 0.6753 - val_acc: 0.5970

Epoch 00001: val_loss improved from inf to 0.67535, saving model to ./model/seed50-01-0.6753.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6743 - acc: 0.6031 - val_loss: 0.6701 - val_acc: 0.5970

Epoch 00002: val_loss improved from 0.67535 to 0.67009, saving model to ./model/seed50-02-0.6701.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6285 - acc: 0.6398 - val_loss: 0.5877 - val_acc: 0.7025

Epoch 00003: val_loss improved from 0.67009 to 0.58767, saving model to ./model/seed50-03-0.5877.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.5010 - acc: 0.7681 - val_loss: 0.5157 - val_acc: 0.7536

Epoch 00004: val_loss improved from 0.58767 to 0.51568, saving model to ./model/seed50-04-0.5157.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

In [44]:
# ----------------------------
seed_num = 50
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/seed50-04-0.5157.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7870550161812297, seed_num = 0
정확도 :0.8019417475728156, seed_num = 1
정확도 :0.7954692556634304, seed_num = 2
정확도 :0.8090614886731392, seed_num = 3
정확도 :0.7954692556634304, seed_num = 4
정확도 :0.8025889967637541, seed_num = 5
정확도 :0.7954692556634304, seed_num = 6
정확도 :0.7877022653721683, seed_num = 7
정확도 :0.8084142394822007, seed_num = 8
정확도 :0.8006472491909385, seed_num = 9
정확도 :0.8129449838187702, seed_num = 10
정확도 :0.8129449838187702, seed_num = 11
정확도 :0.8064724919093851, seed_num = 12
정확도 :0.8116504854368932, seed_num = 13
정확도 :0.8045307443365696, seed_num = 14
정확도 :0.8019417475728156, seed_num = 15
정확도 :0.8116504854368932, seed_num = 16
정확도 :0.7902912621359224, seed_num = 17
정확도 :0.8006472491909385, seed_num = 18
정확도 :0.8019417475728156, seed_num = 19
정확도 :0.8071197411003236, seed_num = 20
정확도 :0.7909385113268609, seed_num = 21
정확도 :0.7941747572815534, seed_num = 22
정확도 :0.7922330097087379, seed_num = 23
정확도 :0.7967637540453074, seed_num = 24
정확도 :0.8148867313915857, seed_num =

# LSTM

## all fit

### (79.9) earlystop

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'allfit3_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 6s 76ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_loss improved from inf to 0.69343, saving model to ./model/allfit3_seed42-01-0.6934.hdf5
Epoch 2/500
46/46 [==============================] - 3s 65ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_loss did not improve from 0.69343
Epoch 3/500
46/46 [==============================] - 3s 66ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_loss did not improve from 0.69343
Epoch 4/500
46/46 [==============================] - 3s 66ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_loss improved from 0.69343 to 0.68479, saving model to ./model/allfit3_seed42-04-0.6848.hdf5
Epoch 5/500
46/46 [==============================] - 3s 65ms/step - loss: 0.5325 - acc: 0.7372 - val_loss: 0.5727 - val_acc: 0.7091

Epoch 00005: val_


Epoch 00040: val_loss did not improve from 0.55251
Epoch 41/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0850 - acc: 0.9789 - val_loss: 1.0211 - val_acc: 0.7308

Epoch 00041: val_loss did not improve from 0.55251
Epoch 42/500
46/46 [==============================] - 3s 66ms/step - loss: 0.0824 - acc: 0.9800 - val_loss: 1.0499 - val_acc: 0.7293

Epoch 00042: val_loss did not improve from 0.55251
Epoch 43/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0812 - acc: 0.9822 - val_loss: 1.0668 - val_acc: 0.7241

Epoch 00043: val_loss did not improve from 0.55251
Epoch 44/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0803 - acc: 0.9821 - val_loss: 1.0820 - val_acc: 0.7262

Epoch 00044: val_loss did not improve from 0.55251
Epoch 45/500
46/46 [==============================] - 3s 67ms/step - loss: 0.0777 - acc: 0.9829 - val_loss: 1.0839 - val_acc: 0.7298

Epoch 00045: val_loss did not improve from 0.55251
Epoch 46/500
46/46 [===

In [2]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/allfit3_seed42-06-0.5525.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

Using TensorFlow backend.


정확도 :0.7980582524271844, seed_num = 0
정확도 :0.8032362459546926, seed_num = 1
정확도 :0.7948220064724919, seed_num = 2
정확도 :0.8064724919093851, seed_num = 3
정확도 :0.8006472491909385, seed_num = 4
정확도 :0.7974110032362459, seed_num = 5
정확도 :0.8025889967637541, seed_num = 6
정확도 :0.7948220064724919, seed_num = 7
정확도 :0.8116504854368932, seed_num = 8
정확도 :0.7922330097087379, seed_num = 9
정확도 :0.8077669902912621, seed_num = 10
정확도 :0.8084142394822007, seed_num = 11
정확도 :0.8084142394822007, seed_num = 12
정확도 :0.8045307443365696, seed_num = 13
정확도 :0.801294498381877, seed_num = 14
정확도 :0.8032362459546926, seed_num = 15
정확도 :0.8051779935275081, seed_num = 16
정확도 :0.7799352750809061, seed_num = 17
정확도 :0.8019417475728156, seed_num = 18
정확도 :0.8071197411003236, seed_num = 19
정확도 :0.7896440129449838, seed_num = 20
정확도 :0.7818770226537217, seed_num = 21
정확도 :0.7928802588996764, seed_num = 22
정확도 :0.7844660194174757, seed_num = 23
정확도 :0.7961165048543689, seed_num = 24
정확도 :0.8122977346278317, seed_num = 

### (79.9) epoch = 500

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random

with tf.device('/device:GPU:0'):
    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    # ---------------------
    seed_num = 42 
    # ---------------------
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'lstm_allfit_noearly-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
    lstm.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 8s 88ms/step - loss: 0.6458 - acc: 0.6511 - val_loss: 0.6934 - val_acc: 0.4865

Epoch 00001: val_loss improved from inf to 0.69343, saving model to ./model/lstm_allfit_noearly-01-0.6934.hdf5
Epoch 2/500
46/46 [==============================] - 3s 75ms/step - loss: 0.6740 - acc: 0.6066 - val_loss: 0.7237 - val_acc: 0.4865

Epoch 00002: val_loss did not improve from 0.69343
Epoch 3/500
46/46 [==============================] - 4s 79ms/step - loss: 0.6651 - acc: 0.6419 - val_loss: 0.7362 - val_acc: 0.4865

Epoch 00003: val_loss did not improve from 0.69343
Epoch 4/500
46/46 [==============================] - 4s 78ms/step - loss: 0.6369 - acc: 0.6518 - val_loss: 0.6848 - val_acc: 0.4865

Epoch 00004: val_loss improved from 0.69343 to 0.68478, saving model to ./model/lstm_allfit_noearly-04-0.6848.hdf5
Epoch 5/500
46/46 [==============================] - 4s 77ms/step - loss: 0.5325 - acc: 0.7367 - val_loss: 0.5728 - val_acc: 0.7091

Epoch 0


Epoch 00040: val_loss did not improve from 0.55254
Epoch 41/500
46/46 [==============================] - 3s 76ms/step - loss: 0.0850 - acc: 0.9810 - val_loss: 1.0421 - val_acc: 0.7293

Epoch 00041: val_loss did not improve from 0.55254
Epoch 42/500
46/46 [==============================] - 4s 79ms/step - loss: 0.0853 - acc: 0.9807 - val_loss: 1.0548 - val_acc: 0.7283

Epoch 00042: val_loss did not improve from 0.55254
Epoch 43/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0831 - acc: 0.9815 - val_loss: 1.0654 - val_acc: 0.7257

Epoch 00043: val_loss did not improve from 0.55254
Epoch 44/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0818 - acc: 0.9821 - val_loss: 1.0804 - val_acc: 0.7241

Epoch 00044: val_loss did not improve from 0.55254
Epoch 45/500
46/46 [==============================] - 3s 74ms/step - loss: 0.0784 - acc: 0.9833 - val_loss: 1.0960 - val_acc: 0.7200

Epoch 00045: val_loss did not improve from 0.55254
Epoch 46/500
46/46 [===

46/46 [==============================] - 4s 77ms/step - loss: 0.0493 - acc: 0.9900 - val_loss: 1.2321 - val_acc: 0.7246

Epoch 00085: val_loss did not improve from 0.55254
Epoch 86/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0459 - acc: 0.9915 - val_loss: 1.2906 - val_acc: 0.7210

Epoch 00086: val_loss did not improve from 0.55254
Epoch 87/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0483 - acc: 0.9907 - val_loss: 1.2797 - val_acc: 0.7195

Epoch 00087: val_loss did not improve from 0.55254
Epoch 88/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0533 - acc: 0.9888 - val_loss: 1.2443 - val_acc: 0.7241

Epoch 00088: val_loss did not improve from 0.55254
Epoch 89/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0489 - acc: 0.9905 - val_loss: 1.2648 - val_acc: 0.7215

Epoch 00089: val_loss did not improve from 0.55254
Epoch 90/500
46/46 [==============================] - 3s 76ms/step - loss: 0.0488 - acc:

46/46 [==============================] - 4s 79ms/step - loss: 0.0219 - acc: 0.9967 - val_loss: 1.7036 - val_acc: 0.7158

Epoch 00173: val_loss did not improve from 0.55254
Epoch 174/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0218 - acc: 0.9967 - val_loss: 1.7084 - val_acc: 0.7164

Epoch 00174: val_loss did not improve from 0.55254
Epoch 175/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0217 - acc: 0.9965 - val_loss: 1.7016 - val_acc: 0.7133

Epoch 00175: val_loss did not improve from 0.55254
Epoch 176/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0223 - acc: 0.9967 - val_loss: 1.7107 - val_acc: 0.7169

Epoch 00176: val_loss did not improve from 0.55254
Epoch 177/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0214 - acc: 0.9967 - val_loss: 1.7188 - val_acc: 0.7153

Epoch 00177: val_loss did not improve from 0.55254
Epoch 178/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0221 -

46/46 [==============================] - 3s 75ms/step - loss: 0.0278 - acc: 0.9940 - val_loss: 1.7476 - val_acc: 0.6977

Epoch 00261: val_loss did not improve from 0.55254
Epoch 262/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0246 - acc: 0.9957 - val_loss: 1.7169 - val_acc: 0.7174

Epoch 00262: val_loss did not improve from 0.55254
Epoch 263/500
46/46 [==============================] - 4s 76ms/step - loss: 0.0233 - acc: 0.9960 - val_loss: 1.7735 - val_acc: 0.7008

Epoch 00263: val_loss did not improve from 0.55254
Epoch 264/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0271 - acc: 0.9950 - val_loss: 1.7120 - val_acc: 0.7050

Epoch 00264: val_loss did not improve from 0.55254
Epoch 265/500
46/46 [==============================] - 3s 75ms/step - loss: 0.0254 - acc: 0.9950 - val_loss: 1.7029 - val_acc: 0.7200

Epoch 00265: val_loss did not improve from 0.55254
Epoch 266/500
46/46 [==============================] - 3s 76ms/step - loss: 0.0226 -

46/46 [==============================] - 4s 77ms/step - loss: 0.0417 - acc: 0.9915 - val_loss: 2.2581 - val_acc: 0.7112

Epoch 00349: val_loss did not improve from 0.55254
Epoch 350/500
46/46 [==============================] - 3s 76ms/step - loss: 0.0669 - acc: 0.9850 - val_loss: 1.5935 - val_acc: 0.7112

Epoch 00350: val_loss did not improve from 0.55254
Epoch 351/500
46/46 [==============================] - 4s 79ms/step - loss: 0.0260 - acc: 0.9943 - val_loss: 1.7733 - val_acc: 0.7158

Epoch 00351: val_loss did not improve from 0.55254
Epoch 352/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0259 - acc: 0.9945 - val_loss: 1.8814 - val_acc: 0.7148

Epoch 00352: val_loss did not improve from 0.55254
Epoch 353/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0265 - acc: 0.9941 - val_loss: 1.7405 - val_acc: 0.7158

Epoch 00353: val_loss did not improve from 0.55254
Epoch 354/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0215 -

46/46 [==============================] - 4s 79ms/step - loss: 0.0140 - acc: 0.9967 - val_loss: 2.1046 - val_acc: 0.7215

Epoch 00437: val_loss did not improve from 0.55254
Epoch 438/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0146 - acc: 0.9967 - val_loss: 2.1118 - val_acc: 0.7210

Epoch 00438: val_loss did not improve from 0.55254
Epoch 439/500
46/46 [==============================] - 3s 76ms/step - loss: 0.0149 - acc: 0.9967 - val_loss: 2.1230 - val_acc: 0.7205

Epoch 00439: val_loss did not improve from 0.55254
Epoch 440/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0149 - acc: 0.9967 - val_loss: 2.1194 - val_acc: 0.7210

Epoch 00440: val_loss did not improve from 0.55254
Epoch 441/500
46/46 [==============================] - 4s 78ms/step - loss: 0.0143 - acc: 0.9967 - val_loss: 2.1260 - val_acc: 0.7220

Epoch 00441: val_loss did not improve from 0.55254
Epoch 442/500
46/46 [==============================] - 4s 77ms/step - loss: 0.0145 -

In [3]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/lstm_allfit_noearly-06-0.5528.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

Using TensorFlow backend.


정확도 :0.7974110032362459, seed_num = 0
정확도 :0.8025889967637541, seed_num = 1
정확도 :0.7948220064724919, seed_num = 2
정확도 :0.8058252427184466, seed_num = 3
정확도 :0.8006472491909385, seed_num = 4
정확도 :0.7974110032362459, seed_num = 5
정확도 :0.8032362459546926, seed_num = 6
정확도 :0.7954692556634304, seed_num = 7
정확도 :0.8116504854368932, seed_num = 8
정확도 :0.7922330097087379, seed_num = 9
정확도 :0.8090614886731392, seed_num = 10
정확도 :0.8084142394822007, seed_num = 11
정확도 :0.8090614886731392, seed_num = 12
정확도 :0.8045307443365696, seed_num = 13
정확도 :0.8006472491909385, seed_num = 14
정확도 :0.8038834951456311, seed_num = 15
정확도 :0.8045307443365696, seed_num = 16
정확도 :0.7799352750809061, seed_num = 17
정확도 :0.8019417475728156, seed_num = 18
정확도 :0.8071197411003236, seed_num = 19
정확도 :0.7896440129449838, seed_num = 20
정확도 :0.7818770226537217, seed_num = 21
정확도 :0.7922330097087379, seed_num = 22
정확도 :0.7844660194174757, seed_num = 23
정확도 :0.7954692556634304, seed_num = 24
정확도 :0.8122977346278317, seed_num =

## not all fit

### (82.6) earlystop - 4069 기준
- x shape : (7727, 10, 4069)

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6654 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66544, saving model to ./model/seed42-01-0.6654.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6690 - acc: 0.6089 - val_loss: 0.6482 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66544 to 0.64818, saving model to ./model/seed42-02-0.6482.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6000 - acc: 0.6713 - val_loss: 0.5525 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.64818 to 0.55246, saving model to ./model/seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.4867 - acc: 0.7793 - val_loss: 0.5088 - val_acc: 0.7613

Epoch 00004: val_loss improved from 0.55246 to 0.50875, saving model to ./model/seed42-04-0.5088.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

37/37 [==============================] - 2s 53ms/step - loss: 0.0637 - acc: 0.9875 - val_loss: 1.1007 - val_acc: 0.7264

Epoch 00044: val_loss did not improve from 0.50213
Epoch 45/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0626 - acc: 0.9871 - val_loss: 1.1301 - val_acc: 0.7290

Epoch 00045: val_loss did not improve from 0.50213
Epoch 46/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0666 - acc: 0.9860 - val_loss: 1.0977 - val_acc: 0.7309

Epoch 00046: val_loss did not improve from 0.50213
Epoch 47/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0645 - acc: 0.9860 - val_loss: 1.1112 - val_acc: 0.7264

Epoch 00047: val_loss did not improve from 0.50213
Epoch 48/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0662 - acc: 0.9853 - val_loss: 1.1206 - val_acc: 0.7283

Epoch 00048: val_loss did not improve from 0.50213
Epoch 49/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0605 - acc:

In [9]:
from keras.models import load_model
best_model = load_model('./model/seed42-05-0.5021.hdf5') 

dic_42={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic_42[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

정확도 :0.8194174757281554, seed_num = 0
정확도 :0.8362459546925566, seed_num = 1
정확도 :0.8207119741100324, seed_num = 2
정확도 :0.8388349514563107, seed_num = 3
정확도 :0.8252427184466019, seed_num = 4
정확도 :0.8401294498381877, seed_num = 5
정확도 :0.8103559870550162, seed_num = 6
정확도 :0.8155339805825242, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8317152103559871, seed_num = 9
정확도 :0.8265372168284789, seed_num = 10
정확도 :0.8310679611650486, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8343042071197411, seed_num = 13
정확도 :0.8381877022653722, seed_num = 14
정확도 :0.8336569579288026, seed_num = 15
정확도 :0.8330097087378641, seed_num = 16
정확도 :0.8174757281553398, seed_num = 17
정확도 :0.8271844660194175, seed_num = 18
정확도 :0.8388349514563107, seed_num = 19
정확도 :0.8284789644012945, seed_num = 20
정확도 :0.8142394822006472, seed_num = 21
정확도 :0.8226537216828479, seed_num = 22
정확도 :0.8090614886731392, seed_num = 23
정확도 :0.8317152103559871, seed_num = 24
정확도 :0.8466019417475729, seed_num = 

In [16]:
# seed = 42의 정확도 df 만들고 평균 확인 => 82.6
df_42 = pd.DataFrame.from_dict(dic_42, orient='index')
df_42.mean()

0    0.826731
dtype: float64

### (82.7) earlystop - 4068 기준
- x shape : (7727, 10, 4068)

In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '4068_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 83ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66548, saving model to ./model/4068_seed42-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 3s 72ms/step - loss: 0.6691 - acc: 0.6089 - val_loss: 0.6488 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66548 to 0.64879, saving model to ./model/4068_seed42-02-0.6488.hdf5
Epoch 3/500
37/37 [==============================] - 3s 72ms/step - loss: 0.6013 - acc: 0.6708 - val_loss: 0.5525 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.64879 to 0.55246, saving model to ./model/4068_seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 67ms/step - loss: 0.4873 - acc: 0.7778 - val_loss: 0.5079 - val_acc: 0.7600

Epoch 00004: val_loss improved from 0.55246 to 0.50789, saving model to ./model/4068_seed42-04-0.5079.hdf5
Epoch 5/500
37/37 [=========================

37/37 [==============================] - 3s 73ms/step - loss: 0.0761 - acc: 0.9830 - val_loss: 1.0363 - val_acc: 0.7296

Epoch 00040: val_loss did not improve from 0.50241
Epoch 41/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0771 - acc: 0.9843 - val_loss: 1.0372 - val_acc: 0.7361

Epoch 00041: val_loss did not improve from 0.50241
Epoch 42/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0711 - acc: 0.9843 - val_loss: 1.0573 - val_acc: 0.7329

Epoch 00042: val_loss did not improve from 0.50241
Epoch 43/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0704 - acc: 0.9845 - val_loss: 1.0824 - val_acc: 0.7225

Epoch 00043: val_loss did not improve from 0.50241
Epoch 44/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0682 - acc: 0.9851 - val_loss: 1.0881 - val_acc: 0.7257

Epoch 00044: val_loss did not improve from 0.50241
Epoch 45/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0717 - acc:

In [8]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/4068_seed42-05-0.5024.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.8213592233009709, seed_num = 0
정확도 :0.8368932038834952, seed_num = 1
정확도 :0.8245954692556634, seed_num = 2
정확도 :0.8375404530744337, seed_num = 3
정확도 :0.8284789644012945, seed_num = 4
정확도 :0.8407766990291262, seed_num = 5
정확도 :0.8116504854368932, seed_num = 6
정확도 :0.8181229773462784, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8310679611650486, seed_num = 9
정확도 :0.8271844660194175, seed_num = 10
정확도 :0.8330097087378641, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8323624595469256, seed_num = 13
정확도 :0.8394822006472492, seed_num = 14
정확도 :0.8330097087378641, seed_num = 15
정확도 :0.8368932038834952, seed_num = 16
정확도 :0.8187702265372169, seed_num = 17
정확도 :0.8233009708737864, seed_num = 18
정확도 :0.8362459546925566, seed_num = 19
정확도 :0.8330097087378641, seed_num = 20
정확도 :0.8129449838187702, seed_num = 21
정확도 :0.8252427184466019, seed_num = 22
정확도 :0.8071197411003236, seed_num = 23
정확도 :0.8349514563106796, seed_num = 24
정확도 :0.8446601941747572, seed_num = 

### (82.6) epoch = 500

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'LSTM_4068_noearly_{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 83ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66548, saving model to ./model/LSTM_4068_noearly_01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6691 - acc: 0.6089 - val_loss: 0.6488 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66548 to 0.64878, saving model to ./model/LSTM_4068_noearly_02-0.6488.hdf5
Epoch 3/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6012 - acc: 0.6704 - val_loss: 0.5525 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.64878 to 0.55252, saving model to ./model/LSTM_4068_noearly_03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4872 - acc: 0.7780 - val_loss: 0.5080 - val_acc: 0.7607

Epoch 00004: val_loss improved from 0.55252 to 0.50803, saving model to ./model/LSTM_4068_noearly_04-0.5080.hdf5
Epoch 5/500
37/37 [=

37/37 [==============================] - 3s 73ms/step - loss: 0.0839 - acc: 0.9810 - val_loss: 1.0260 - val_acc: 0.7303

Epoch 00040: val_loss did not improve from 0.50262
Epoch 41/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0845 - acc: 0.9814 - val_loss: 1.0297 - val_acc: 0.7329

Epoch 00041: val_loss did not improve from 0.50262
Epoch 42/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0846 - acc: 0.9791 - val_loss: 1.0267 - val_acc: 0.7367

Epoch 00042: val_loss did not improve from 0.50262
Epoch 43/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1032 - acc: 0.9709 - val_loss: 1.0086 - val_acc: 0.7342

Epoch 00043: val_loss did not improve from 0.50262
Epoch 44/500
37/37 [==============================] - 3s 72ms/step - loss: 0.1172 - acc: 0.9642 - val_loss: 1.0610 - val_acc: 0.7264

Epoch 00044: val_loss did not improve from 0.50262
Epoch 45/500
37/37 [==============================] - 3s 71ms/step - loss: 0.1461 - acc:

37/37 [==============================] - 3s 71ms/step - loss: 0.0588 - acc: 0.9862 - val_loss: 1.2148 - val_acc: 0.7212

Epoch 00084: val_loss did not improve from 0.50262
Epoch 85/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0497 - acc: 0.9899 - val_loss: 1.1666 - val_acc: 0.7322

Epoch 00085: val_loss did not improve from 0.50262
Epoch 86/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0486 - acc: 0.9905 - val_loss: 1.1760 - val_acc: 0.7296

Epoch 00086: val_loss did not improve from 0.50262
Epoch 87/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0467 - acc: 0.9918 - val_loss: 1.1816 - val_acc: 0.7348

Epoch 00087: val_loss did not improve from 0.50262
Epoch 88/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0473 - acc: 0.9918 - val_loss: 1.2217 - val_acc: 0.7335

Epoch 00088: val_loss did not improve from 0.50262
Epoch 89/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0489 - acc:

37/37 [==============================] - 3s 70ms/step - loss: 0.0413 - acc: 0.9925 - val_loss: 1.3548 - val_acc: 0.7212

Epoch 00128: val_loss did not improve from 0.50262
Epoch 129/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0543 - acc: 0.9875 - val_loss: 1.2857 - val_acc: 0.7270

Epoch 00129: val_loss did not improve from 0.50262
Epoch 130/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0573 - acc: 0.9866 - val_loss: 1.2352 - val_acc: 0.7354

Epoch 00130: val_loss did not improve from 0.50262
Epoch 131/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0628 - acc: 0.9855 - val_loss: 1.2367 - val_acc: 0.7277

Epoch 00131: val_loss did not improve from 0.50262
Epoch 132/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0488 - acc: 0.9899 - val_loss: 1.2540 - val_acc: 0.7264

Epoch 00132: val_loss did not improve from 0.50262
Epoch 133/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0465 -

37/37 [==============================] - 3s 71ms/step - loss: 0.0339 - acc: 0.9948 - val_loss: 1.3699 - val_acc: 0.7335

Epoch 00172: val_loss did not improve from 0.50262
Epoch 173/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0321 - acc: 0.9948 - val_loss: 1.3764 - val_acc: 0.7303

Epoch 00173: val_loss did not improve from 0.50262
Epoch 174/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0335 - acc: 0.9948 - val_loss: 1.3772 - val_acc: 0.7303

Epoch 00174: val_loss did not improve from 0.50262
Epoch 175/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0332 - acc: 0.9946 - val_loss: 1.3973 - val_acc: 0.7264

Epoch 00175: val_loss did not improve from 0.50262
Epoch 176/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0337 - acc: 0.9948 - val_loss: 1.3832 - val_acc: 0.7296

Epoch 00176: val_loss did not improve from 0.50262
Epoch 177/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0334 -

...

37/37 [==============================] - 3s 70ms/step - loss: 0.0317 - acc: 0.9948 - val_loss: 1.4137 - val_acc: 0.7225

Epoch 00304: val_loss did not improve from 0.50262
Epoch 305/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0320 - acc: 0.9948 - val_loss: 1.4159 - val_acc: 0.7232

Epoch 00305: val_loss did not improve from 0.50262
Epoch 306/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0327 - acc: 0.9948 - val_loss: 1.4199 - val_acc: 0.7206

Epoch 00306: val_loss did not improve from 0.50262
Epoch 307/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0316 - acc: 0.9948 - val_loss: 1.4183 - val_acc: 0.7219

Epoch 00307: val_loss did not improve from 0.50262
Epoch 308/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0309 - acc: 0.9948 - val_loss: 1.4072 - val_acc: 0.7212

Epoch 00308: val_loss did not improve from 0.50262
Epoch 309/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0326 -

37/37 [==============================] - 3s 71ms/step - loss: 0.0288 - acc: 0.9950 - val_loss: 1.5097 - val_acc: 0.7141

Epoch 00348: val_loss did not improve from 0.50262
Epoch 349/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0299 - acc: 0.9950 - val_loss: 1.5164 - val_acc: 0.7147

Epoch 00349: val_loss did not improve from 0.50262
Epoch 350/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0292 - acc: 0.9950 - val_loss: 1.5096 - val_acc: 0.7135

Epoch 00350: val_loss did not improve from 0.50262
Epoch 351/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0288 - acc: 0.9950 - val_loss: 1.5114 - val_acc: 0.7128

Epoch 00351: val_loss did not improve from 0.50262
Epoch 352/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0299 - acc: 0.9950 - val_loss: 1.5151 - val_acc: 0.7154

Epoch 00352: val_loss did not improve from 0.50262
Epoch 353/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0303 -

37/37 [==============================] - 3s 72ms/step - loss: 0.0272 - acc: 0.9950 - val_loss: 1.5697 - val_acc: 0.7135

Epoch 00392: val_loss did not improve from 0.50262
Epoch 393/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0264 - acc: 0.9950 - val_loss: 1.5816 - val_acc: 0.7122

Epoch 00393: val_loss did not improve from 0.50262
Epoch 394/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0281 - acc: 0.9950 - val_loss: 1.5817 - val_acc: 0.7115

Epoch 00394: val_loss did not improve from 0.50262
Epoch 395/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0277 - acc: 0.9950 - val_loss: 1.5825 - val_acc: 0.7102

Epoch 00395: val_loss did not improve from 0.50262
Epoch 396/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0267 - acc: 0.9950 - val_loss: 1.5781 - val_acc: 0.7109

Epoch 00396: val_loss did not improve from 0.50262
Epoch 397/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0261 -

37/37 [==============================] - 3s 71ms/step - loss: 0.0251 - acc: 0.9950 - val_loss: 1.6525 - val_acc: 0.7063

Epoch 00436: val_loss did not improve from 0.50262
Epoch 437/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0251 - acc: 0.9950 - val_loss: 1.6500 - val_acc: 0.7083

Epoch 00437: val_loss did not improve from 0.50262
Epoch 438/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0260 - acc: 0.9950 - val_loss: 1.6523 - val_acc: 0.7063

Epoch 00438: val_loss did not improve from 0.50262
Epoch 439/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0248 - acc: 0.9950 - val_loss: 1.6560 - val_acc: 0.7076

Epoch 00439: val_loss did not improve from 0.50262
Epoch 440/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0268 - acc: 0.9950 - val_loss: 1.6514 - val_acc: 0.7076

Epoch 00440: val_loss did not improve from 0.50262
Epoch 441/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0257 -

37/37 [==============================] - 3s 70ms/step - loss: 0.0269 - acc: 0.9946 - val_loss: 1.4323 - val_acc: 0.7277

Epoch 00480: val_loss did not improve from 0.50262
Epoch 481/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0254 - acc: 0.9948 - val_loss: 1.4791 - val_acc: 0.7186

Epoch 00481: val_loss did not improve from 0.50262
Epoch 482/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0255 - acc: 0.9946 - val_loss: 1.5167 - val_acc: 0.7180

Epoch 00482: val_loss did not improve from 0.50262
Epoch 483/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0250 - acc: 0.9950 - val_loss: 1.5276 - val_acc: 0.7225

Epoch 00483: val_loss did not improve from 0.50262
Epoch 484/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0250 - acc: 0.9950 - val_loss: 1.5459 - val_acc: 0.7225

Epoch 00484: val_loss did not improve from 0.50262
Epoch 485/500
37/37 [==============================] - 3s 71ms/step - loss: 0.0246 -

In [6]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:0'):

    from keras.models import load_model
    best_model = load_model('./model/LSTM_4068_noearly_05-0.5026.hdf5') # 수동으로 확인

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.8220064724919094, seed_num = 0
정확도 :0.8368932038834952, seed_num = 1
정확도 :0.8239482200647249, seed_num = 2
정확도 :0.8375404530744337, seed_num = 3
정확도 :0.8265372168284789, seed_num = 4
정확도 :0.8407766990291262, seed_num = 5
정확도 :0.8110032362459547, seed_num = 6
정확도 :0.8181229773462784, seed_num = 7
정확도 :0.8271844660194175, seed_num = 8
정확도 :0.8317152103559871, seed_num = 9
정확도 :0.8265372168284789, seed_num = 10
정확도 :0.8330097087378641, seed_num = 11
정확도 :0.8355987055016181, seed_num = 12
정확도 :0.8330097087378641, seed_num = 13
정확도 :0.8388349514563107, seed_num = 14
정확도 :0.8330097087378641, seed_num = 15
정확도 :0.8381877022653722, seed_num = 16
정확도 :0.8187702265372169, seed_num = 17
정확도 :0.8239482200647249, seed_num = 18
정확도 :0.8368932038834952, seed_num = 19
정확도 :0.8330097087378641, seed_num = 20
정확도 :0.8122977346278317, seed_num = 21
정확도 :0.8245954692556634, seed_num = 22
정확도 :0.8071197411003236, seed_num = 23
정확도 :0.8343042071197411, seed_num = 24
정확도 :0.8453074433656957, seed_num =

# CuDNNLSTM

## all fit

### (60.8) earlystop

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (x.shape[1],x.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

with tf.device('/device:GPU:1'):
    model = cudnn_lstm_model()

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '4068_allfit-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=False)
    model.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 2s 24ms/step - loss: 0.6569 - accuracy: 0.6459 - val_loss: 0.7095 - val_accuracy: 0.4865

Epoch 00001: val_loss improved from inf to 0.70946, saving model to ./model/CuDNNLSTM/4068_allfit_p80-01-0.7095.hdf5
Epoch 2/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6757 - accuracy: 0.6088 - val_loss: 0.7536 - val_accuracy: 0.4865

Epoch 00002: val_loss did not improve from 0.70946
Epoch 3/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6652 - accuracy: 0.6385 - val_loss: 0.7482 - val_accuracy: 0.4865

Epoch 00003: val_loss did not improve from 0.70946
Epoch 4/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6636 - accuracy: 0.6433 - val_loss: 0.7505 - val_accuracy: 0.4865

Epoch 00004: val_loss did not improve from 0.70946
Epoch 5/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6631 - accuracy: 0.6406 - val_loss: 0.7496 - val_accuracy: 0.4865

Epoch 00005: va


Epoch 00042: val_loss did not improve from 0.70946
Epoch 43/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6545 - accuracy: 0.6513 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00043: val_loss did not improve from 0.70946
Epoch 44/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6533 - accuracy: 0.6513 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00044: val_loss did not improve from 0.70946
Epoch 45/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6577 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00045: val_loss did not improve from 0.70946
Epoch 46/500
46/46 [==============================] - 1s 14ms/step - loss: 0.6528 - accuracy: 0.6504 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00046: val_loss did not improve from 0.70946
Epoch 47/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6509 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00047: val_loss di

In [15]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/4068_allfit-01-0.7095.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.6045307443365696, seed_num = 0
정확도 :0.6051779935275081, seed_num = 1
정확도 :0.6110032362459547, seed_num = 2
정확도 :0.602588996763754, seed_num = 3
정확도 :0.6097087378640776, seed_num = 4
정확도 :0.6110032362459547, seed_num = 5
정확도 :0.6110032362459547, seed_num = 6
정확도 :0.5928802588996763, seed_num = 7
정확도 :0.6226537216828479, seed_num = 8
정확도 :0.6129449838187703, seed_num = 9
정확도 :0.5909385113268608, seed_num = 10
정확도 :0.6297734627831715, seed_num = 11
정확도 :0.601294498381877, seed_num = 12
정확도 :0.598705501618123, seed_num = 13
정확도 :0.6071197411003236, seed_num = 14
정확도 :0.6135922330097088, seed_num = 15
정확도 :0.6330097087378641, seed_num = 16
정확도 :0.598705501618123, seed_num = 17
정확도 :0.6045307443365696, seed_num = 18
정확도 :0.596116504854369, seed_num = 19
정확도 :0.6187702265372168, seed_num = 20
정확도 :0.6135922330097088, seed_num = 21
정확도 :0.6, seed_num = 22
정확도 :0.6032362459546926, seed_num = 23
정확도 :0.6032362459546926, seed_num = 24
정확도 :0.6071197411003236, seed_num = 25
정확도 :0.595469255

### (83.9) epoch = 500

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (x.shape[1],x.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

with tf.device('/device:GPU:0'):
    model = cudnn_lstm_model()

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '4068_allfit_noearly-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    model.fit(x, y, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
46/46 [==============================] - 3s 22ms/step - loss: 0.6569 - accuracy: 0.6459 - val_loss: 0.7095 - val_accuracy: 0.4865

Epoch 00001: val_loss improved from inf to 0.70946, saving model to ./model/CuDNNLSTM/4068_allfit_noearly-01-0.7095.hdf5
Epoch 2/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6757 - accuracy: 0.6088 - val_loss: 0.7536 - val_accuracy: 0.4865

Epoch 00002: val_loss did not improve from 0.70946
Epoch 3/500
46/46 [==============================] - 0s 11ms/step - loss: 0.6652 - accuracy: 0.6385 - val_loss: 0.7481 - val_accuracy: 0.4865

Epoch 00003: val_loss did not improve from 0.70946
Epoch 4/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6636 - accuracy: 0.6433 - val_loss: 0.7505 - val_accuracy: 0.4865

Epoch 00004: val_loss did not improve from 0.70946
Epoch 5/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6631 - accuracy: 0.6406 - val_loss: 0.7496 - val_accuracy: 0.4865

Epoch 00005


Epoch 00042: val_loss did not improve from 0.70946
Epoch 43/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6545 - accuracy: 0.6513 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00043: val_loss did not improve from 0.70946
Epoch 44/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6533 - accuracy: 0.6513 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00044: val_loss did not improve from 0.70946
Epoch 45/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6577 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00045: val_loss did not improve from 0.70946
Epoch 46/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6528 - accuracy: 0.6504 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00046: val_loss did not improve from 0.70946
Epoch 47/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6509 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00047: val_loss di

46/46 [==============================] - 1s 13ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00085: val_loss did not improve from 0.70946
Epoch 86/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6529 - accuracy: 0.6518 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00086: val_loss did not improve from 0.70946
Epoch 87/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00087: val_loss did not improve from 0.70946
Epoch 88/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6515 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00088: val_loss did not improve from 0.70946
Epoch 89/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6508 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00089: val_loss did not improve from 0.70946
Epoch 90/500
46/46 [==================

46/46 [==============================] - 1s 13ms/step - loss: 0.6510 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00127: val_loss did not improve from 0.70946
Epoch 128/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7477 - val_accuracy: 0.4865

Epoch 00128: val_loss did not improve from 0.70946
Epoch 129/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6504 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00129: val_loss did not improve from 0.70946
Epoch 130/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00130: val_loss did not improve from 0.70946
Epoch 131/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6502 - accuracy: 0.6519 - val_loss: 0.7474 - val_accuracy: 0.4865

Epoch 00131: val_loss did not improve from 0.70946
Epoch 132/500
46/46 [=============

46/46 [==============================] - 1s 12ms/step - loss: 0.6490 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00169: val_loss did not improve from 0.70946
Epoch 170/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6495 - accuracy: 0.6519 - val_loss: 0.7467 - val_accuracy: 0.4865

Epoch 00170: val_loss did not improve from 0.70946
Epoch 171/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7466 - val_accuracy: 0.4865

Epoch 00171: val_loss did not improve from 0.70946
Epoch 172/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6501 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00172: val_loss did not improve from 0.70946
Epoch 173/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00173: val_loss did not improve from 0.70946
Epoch 174/500
46/46 [=============

...

46/46 [==============================] - 1s 12ms/step - loss: 0.6489 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00295: val_loss did not improve from 0.70946
Epoch 296/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6495 - accuracy: 0.6519 - val_loss: 0.7464 - val_accuracy: 0.4865

Epoch 00296: val_loss did not improve from 0.70946
Epoch 297/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6489 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00297: val_loss did not improve from 0.70946
Epoch 298/500
46/46 [==============================] - 1s 13ms/step - loss: 0.6490 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00298: val_loss did not improve from 0.70946
Epoch 299/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6486 - accuracy: 0.6519 - val_loss: 0.7471 - val_accuracy: 0.4865

Epoch 00299: val_loss did not improve from 0.70946
Epoch 300/500
46/46 [=============

46/46 [==============================] - 1s 13ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00337: val_loss did not improve from 0.70946
Epoch 338/500
46/46 [==============================] - 1s 11ms/step - loss: 0.6488 - accuracy: 0.6519 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00338: val_loss did not improve from 0.70946
Epoch 339/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6487 - accuracy: 0.6519 - val_loss: 0.7474 - val_accuracy: 0.4865

Epoch 00339: val_loss did not improve from 0.70946
Epoch 340/500
46/46 [==============================] - 0s 11ms/step - loss: 0.6489 - accuracy: 0.6519 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00340: val_loss did not improve from 0.70946
Epoch 341/500
46/46 [==============================] - 1s 12ms/step - loss: 0.6491 - accuracy: 0.6519 - val_loss: 0.7466 - val_accuracy: 0.4865

Epoch 00341: val_loss did not improve from 0.70946
Epoch 342/500
46/46 [=============

46/46 [==============================] - 1s 11ms/step - loss: 0.3584 - accuracy: 0.8937 - val_loss: 0.6423 - val_accuracy: 0.7091

Epoch 00375: val_loss did not improve from 0.62318
Epoch 376/500
46/46 [==============================] - 1s 12ms/step - loss: 0.3514 - accuracy: 0.8991 - val_loss: 0.6418 - val_accuracy: 0.7096

Epoch 00376: val_loss did not improve from 0.62318
Epoch 377/500
46/46 [==============================] - 1s 11ms/step - loss: 0.3359 - accuracy: 0.9020 - val_loss: 0.6485 - val_accuracy: 0.7045

Epoch 00377: val_loss did not improve from 0.62318
Epoch 378/500
46/46 [==============================] - 1s 12ms/step - loss: 0.3397 - accuracy: 0.9035 - val_loss: 0.6623 - val_accuracy: 0.7034

Epoch 00378: val_loss did not improve from 0.62318
Epoch 379/500
46/46 [==============================] - 1s 12ms/step - loss: 0.3320 - accuracy: 0.9060 - val_loss: 0.6614 - val_accuracy: 0.7008

Epoch 00379: val_loss did not improve from 0.62318
Epoch 380/500
46/46 [=============

46/46 [==============================] - 1s 11ms/step - loss: 0.2537 - accuracy: 0.9384 - val_loss: 0.7848 - val_accuracy: 0.7029

Epoch 00417: val_loss did not improve from 0.62318
Epoch 418/500
46/46 [==============================] - 1s 13ms/step - loss: 0.2613 - accuracy: 0.9363 - val_loss: 0.7933 - val_accuracy: 0.7039

Epoch 00418: val_loss did not improve from 0.62318
Epoch 419/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2556 - accuracy: 0.9367 - val_loss: 0.7924 - val_accuracy: 0.7045

Epoch 00419: val_loss did not improve from 0.62318
Epoch 420/500
46/46 [==============================] - 1s 11ms/step - loss: 0.2533 - accuracy: 0.9377 - val_loss: 0.7913 - val_accuracy: 0.7060

Epoch 00420: val_loss did not improve from 0.62318
Epoch 421/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2517 - accuracy: 0.9377 - val_loss: 0.8001 - val_accuracy: 0.6967

Epoch 00421: val_loss did not improve from 0.62318
Epoch 422/500
46/46 [=============

46/46 [==============================] - 1s 11ms/step - loss: 0.2335 - accuracy: 0.9453 - val_loss: 0.8612 - val_accuracy: 0.7019

Epoch 00459: val_loss did not improve from 0.62318
Epoch 460/500
46/46 [==============================] - 1s 13ms/step - loss: 0.2383 - accuracy: 0.9420 - val_loss: 0.8878 - val_accuracy: 0.6874

Epoch 00460: val_loss did not improve from 0.62318
Epoch 461/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2329 - accuracy: 0.9446 - val_loss: 0.8826 - val_accuracy: 0.6925

Epoch 00461: val_loss did not improve from 0.62318
Epoch 462/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2299 - accuracy: 0.9460 - val_loss: 0.8853 - val_accuracy: 0.6915

Epoch 00462: val_loss did not improve from 0.62318
Epoch 463/500
46/46 [==============================] - 1s 12ms/step - loss: 0.2221 - accuracy: 0.9467 - val_loss: 0.8850 - val_accuracy: 0.6951

Epoch 00463: val_loss did not improve from 0.62318
Epoch 464/500
46/46 [=============

In [5]:
from sklearn import metrics 

import random
# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:1'):

    from keras.models import load_model

    best_model = load_model('./model/CuDNNLSTM/4068_allfit_noearly-367-0.6232.hdf5') 

    dic={}
    for seed in range(0, 50):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

Using TensorFlow backend.


정확도 :0.8310679611650486, seed_num = 0
정확도 :0.8453074433656957, seed_num = 1
정확도 :0.8401294498381877, seed_num = 2
정확도 :0.8414239482200647, seed_num = 3
정확도 :0.8355987055016181, seed_num = 4
정확도 :0.8420711974110032, seed_num = 5
정확도 :0.8433656957928802, seed_num = 6
정확도 :0.8537216828478964, seed_num = 7
정확도 :0.8388349514563107, seed_num = 8
정확도 :0.8284789644012945, seed_num = 9
정확도 :0.8537216828478964, seed_num = 10
정확도 :0.8427184466019417, seed_num = 11
정확도 :0.8446601941747572, seed_num = 12
정확도 :0.8414239482200647, seed_num = 13
정확도 :0.8491909385113269, seed_num = 14
정확도 :0.8394822006472492, seed_num = 15
정확도 :0.8401294498381877, seed_num = 16
정확도 :0.83042071197411, seed_num = 17
정확도 :0.8284789644012945, seed_num = 18
정확도 :0.8433656957928802, seed_num = 19
정확도 :0.8362459546925566, seed_num = 20
정확도 :0.8181229773462784, seed_num = 21
정확도 :0.8407766990291262, seed_num = 22
정확도 :0.8407766990291262, seed_num = 23
정확도 :0.8330097087378641, seed_num = 24
정확도 :0.8511326860841424, seed_num = 2

### (84.8) epoch = 1000

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (x.shape[1],x.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

with tf.device('/device:GPU:0'):
    model = cudnn_lstm_model()

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'e1000_allfit-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    model.fit(x, y, validation_split=0.25, batch_size=128, epochs=1000,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/1000
46/46 [==============================] - 2s 22ms/step - loss: 0.6569 - accuracy: 0.6459 - val_loss: 0.7095 - val_accuracy: 0.4865

Epoch 00001: val_loss improved from inf to 0.70946, saving model to ./model/CuDNNLSTM/e1000_allfit-01-0.7095.hdf5
Epoch 2/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6757 - accuracy: 0.6088 - val_loss: 0.7536 - val_accuracy: 0.4865

Epoch 00002: val_loss did not improve from 0.70946
Epoch 3/1000
46/46 [==============================] - 0s 11ms/step - loss: 0.6652 - accuracy: 0.6385 - val_loss: 0.7482 - val_accuracy: 0.4865

Epoch 00003: val_loss did not improve from 0.70946
Epoch 4/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6636 - accuracy: 0.6433 - val_loss: 0.7505 - val_accuracy: 0.4865

Epoch 00004: val_loss did not improve from 0.70946
Epoch 5/1000
46/46 [==============================] - 0s 11ms/step - loss: 0.6631 - accuracy: 0.6406 - val_loss: 0.7496 - val_accuracy: 0.4865

Epoch 00005: 


Epoch 00042: val_loss did not improve from 0.70946
Epoch 43/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.6545 - accuracy: 0.6513 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00043: val_loss did not improve from 0.70946
Epoch 44/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6533 - accuracy: 0.6513 - val_loss: 0.7472 - val_accuracy: 0.4865

Epoch 00044: val_loss did not improve from 0.70946
Epoch 45/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6577 - accuracy: 0.6519 - val_loss: 0.7469 - val_accuracy: 0.4865

Epoch 00045: val_loss did not improve from 0.70946
Epoch 46/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6528 - accuracy: 0.6504 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00046: val_loss did not improve from 0.70946
Epoch 47/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6533 - accuracy: 0.6509 - val_loss: 0.7479 - val_accuracy: 0.4865

Epoch 00047: val_lo


Epoch 00084: val_loss did not improve from 0.70946
Epoch 85/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00085: val_loss did not improve from 0.70946
Epoch 86/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.6529 - accuracy: 0.6518 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00086: val_loss did not improve from 0.70946
Epoch 87/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6511 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00087: val_loss did not improve from 0.70946
Epoch 88/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6515 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00088: val_loss did not improve from 0.70946
Epoch 89/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6508 - accuracy: 0.6519 - val_loss: 0.7470 - val_accuracy: 0.4865

Epoch 00089: val_lo

46/46 [==============================] - 0s 11ms/step - loss: 0.6500 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00126: val_loss did not improve from 0.70946
Epoch 127/1000
46/46 [==============================] - 0s 11ms/step - loss: 0.6510 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00127: val_loss did not improve from 0.70946
Epoch 128/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7477 - val_accuracy: 0.4865

Epoch 00128: val_loss did not improve from 0.70946
Epoch 129/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6504 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00129: val_loss did not improve from 0.70946
Epoch 130/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.6494 - accuracy: 0.6519 - val_loss: 0.7476 - val_accuracy: 0.4865

Epoch 00130: val_loss did not improve from 0.70946
Epoch 131/1000
46/46 [========


Epoch 00167: val_loss did not improve from 0.70946
Epoch 168/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.6496 - accuracy: 0.6519 - val_loss: 0.7465 - val_accuracy: 0.4865

Epoch 00168: val_loss did not improve from 0.70946
Epoch 169/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6490 - accuracy: 0.6519 - val_loss: 0.7473 - val_accuracy: 0.4865

Epoch 00169: val_loss did not improve from 0.70946
Epoch 170/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6495 - accuracy: 0.6519 - val_loss: 0.7467 - val_accuracy: 0.4865

Epoch 00170: val_loss did not improve from 0.70946
Epoch 171/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6493 - accuracy: 0.6519 - val_loss: 0.7466 - val_accuracy: 0.4865

Epoch 00171: val_loss did not improve from 0.70946
Epoch 172/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.6501 - accuracy: 0.6519 - val_loss: 0.7468 - val_accuracy: 0.4865

Epoch 00172: v

...


Epoch 00826: val_loss did not improve from 0.63924
Epoch 827/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0515 - accuracy: 0.9905 - val_loss: 1.5822 - val_accuracy: 0.7060

Epoch 00827: val_loss did not improve from 0.63924
Epoch 828/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.0535 - accuracy: 0.9914 - val_loss: 1.5818 - val_accuracy: 0.6925

Epoch 00828: val_loss did not improve from 0.63924
Epoch 829/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0619 - accuracy: 0.9886 - val_loss: 1.5173 - val_accuracy: 0.6957

Epoch 00829: val_loss did not improve from 0.63924
Epoch 830/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.0506 - accuracy: 0.9905 - val_loss: 1.5781 - val_accuracy: 0.7045

Epoch 00830: val_loss did not improve from 0.63924
Epoch 831/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0497 - accuracy: 0.9912 - val_loss: 1.4873 - val_accuracy: 0.7050

Epoch 00831: v

46/46 [==============================] - 1s 13ms/step - loss: 0.0449 - accuracy: 0.9931 - val_loss: 1.5569 - val_accuracy: 0.7024

Epoch 00868: val_loss did not improve from 0.63924
Epoch 869/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0436 - accuracy: 0.9929 - val_loss: 1.5589 - val_accuracy: 0.7065

Epoch 00869: val_loss did not improve from 0.63924
Epoch 870/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0450 - accuracy: 0.9926 - val_loss: 1.5828 - val_accuracy: 0.7039

Epoch 00870: val_loss did not improve from 0.63924
Epoch 871/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0455 - accuracy: 0.9929 - val_loss: 1.5756 - val_accuracy: 0.6982

Epoch 00871: val_loss did not improve from 0.63924
Epoch 872/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0432 - accuracy: 0.9928 - val_loss: 1.5320 - val_accuracy: 0.7122

Epoch 00872: val_loss did not improve from 0.63924
Epoch 873/1000
46/46 [========


Epoch 00909: val_loss did not improve from 0.63924
Epoch 910/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0608 - accuracy: 0.9890 - val_loss: 1.5065 - val_accuracy: 0.7127

Epoch 00910: val_loss did not improve from 0.63924
Epoch 911/1000
46/46 [==============================] - 1s 13ms/step - loss: 0.0445 - accuracy: 0.9929 - val_loss: 1.4850 - val_accuracy: 0.7101

Epoch 00911: val_loss did not improve from 0.63924
Epoch 912/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0456 - accuracy: 0.9922 - val_loss: 1.4779 - val_accuracy: 0.7112

Epoch 00912: val_loss did not improve from 0.63924
Epoch 913/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0446 - accuracy: 0.9924 - val_loss: 1.5055 - val_accuracy: 0.7117

Epoch 00913: val_loss did not improve from 0.63924
Epoch 914/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0458 - accuracy: 0.9919 - val_loss: 1.5105 - val_accuracy: 0.7112

Epoch 00914: v

46/46 [==============================] - 1s 12ms/step - loss: 0.0374 - accuracy: 0.9934 - val_loss: 1.6120 - val_accuracy: 0.7024

Epoch 00951: val_loss did not improve from 0.63924
Epoch 952/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0396 - accuracy: 0.9940 - val_loss: 1.6069 - val_accuracy: 0.7034

Epoch 00952: val_loss did not improve from 0.63924
Epoch 953/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0359 - accuracy: 0.9947 - val_loss: 1.6185 - val_accuracy: 0.7045

Epoch 00953: val_loss did not improve from 0.63924
Epoch 954/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0368 - accuracy: 0.9936 - val_loss: 1.6102 - val_accuracy: 0.7065

Epoch 00954: val_loss did not improve from 0.63924
Epoch 955/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0375 - accuracy: 0.9945 - val_loss: 1.6073 - val_accuracy: 0.7060

Epoch 00955: val_loss did not improve from 0.63924
Epoch 956/1000
46/46 [========


Epoch 00992: val_loss did not improve from 0.63924
Epoch 993/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0682 - accuracy: 0.9831 - val_loss: 1.6071 - val_accuracy: 0.7003

Epoch 00993: val_loss did not improve from 0.63924
Epoch 994/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0470 - accuracy: 0.9922 - val_loss: 1.5455 - val_accuracy: 0.7070

Epoch 00994: val_loss did not improve from 0.63924
Epoch 995/1000
46/46 [==============================] - 1s 12ms/step - loss: 0.0482 - accuracy: 0.9915 - val_loss: 1.6288 - val_accuracy: 0.7008

Epoch 00995: val_loss did not improve from 0.63924
Epoch 996/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0447 - accuracy: 0.9926 - val_loss: 1.5952 - val_accuracy: 0.7024

Epoch 00996: val_loss did not improve from 0.63924
Epoch 997/1000
46/46 [==============================] - 1s 11ms/step - loss: 0.0415 - accuracy: 0.9929 - val_loss: 1.5963 - val_accuracy: 0.7008

Epoch 00997: v

In [14]:
from sklearn import metrics 
from tqdm.notebook import tqdm
import random

# ----------------------------
seed_num = 42
# ----------------------------

with tf.device('/device:GPU:1'):

    from keras.models import load_model

    best_model = load_model('./model/CuDNNLSTM/e1000_allfit-347-0.6392.hdf5') 

    dic={}
    for seed in tqdm(range(0, 50)):
        random.seed(seed)

        x = np.load('/project/LSH/x_(7727,10,4068).npy')
        y = np.load('/project/LSH/y_(7727,1).npy')

        idx = list(range(len(x)))
        random.shuffle(idx)

        i = round(x.shape[0]*0.8)
        X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
        X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

        pred = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
        pred[pred>0.5]=1
        pred[pred<=0.5]=0
        acc = metrics.accuracy_score(y_test, pred)
        dic[seed]=acc
        print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(dic, orient='index')
    print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

  0%|          | 0/50 [00:00<?, ?it/s]

정확도 :0.8517799352750809, seed_num = 0
정확도 :0.8485436893203884, seed_num = 1
정확도 :0.856957928802589, seed_num = 2
정확도 :0.8498381877022654, seed_num = 3
정확도 :0.8478964401294499, seed_num = 4
정확도 :0.8498381877022654, seed_num = 5
정확도 :0.8420711974110032, seed_num = 6
정확도 :0.8588996763754045, seed_num = 7
정확도 :0.8466019417475729, seed_num = 8
정확도 :0.8420711974110032, seed_num = 9
정확도 :0.8517799352750809, seed_num = 10
정확도 :0.8524271844660194, seed_num = 11
정확도 :0.8504854368932039, seed_num = 12
정확도 :0.8504854368932039, seed_num = 13
정확도 :0.8440129449838187, seed_num = 14
정확도 :0.8576051779935275, seed_num = 15
정확도 :0.8407766990291262, seed_num = 16
정확도 :0.8394822006472492, seed_num = 17
정확도 :0.8433656957928802, seed_num = 18
정확도 :0.8543689320388349, seed_num = 19
정확도 :0.8491909385113269, seed_num = 20
정확도 :0.8381877022653722, seed_num = 21
정확도 :0.8459546925566344, seed_num = 22
정확도 :0.8446601941747572, seed_num = 23
정확도 :0.8394822006472492, seed_num = 24
정확도 :0.8588996763754045, seed_num = 

## not all fit

### (76.7) earlystop

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42
# ---------------------
tf.random.set_seed(seed_num)


def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(units=64, return_sequences=True))
    model.add(Activation('hard_sigmoid'))
    model.add(CuDNNLSTM(units=32, return_sequences=False))
    model.add(Activation('hard_sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
    
    return model

model = cudnn_lstm_model()

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '4068_notallfit-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=False)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 2s 23ms/step - loss: 0.6849 - accuracy: 0.5887 - val_loss: 0.6655 - val_accuracy: 0.6177

Epoch 00001: val_loss improved from inf to 0.66551, saving model to ./model/CuDNNLSTM/4068_notallfit-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6810 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00002: val_loss improved from 0.66551 to 0.66520, saving model to ./model/CuDNNLSTM/4068_notallfit-02-0.6652.hdf5
Epoch 3/500
37/37 [==============================] - 0s 13ms/step - loss: 0.6814 - accuracy: 0.5951 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00003: val_loss did not improve from 0.66520
Epoch 4/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6772 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00004: val_loss improved from 0.66520 to 0.66517, saving model to ./model/CuDNNLSTM/4068_notallfit-04-0.6652.hdf5
Epoch 5/500
37/37 [==

37/37 [==============================] - 0s 13ms/step - loss: 0.3132 - accuracy: 0.9085 - val_loss: 0.6783 - val_accuracy: 0.7277

Epoch 00039: val_loss did not improve from 0.58819
Epoch 40/500
37/37 [==============================] - 0s 12ms/step - loss: 0.2987 - accuracy: 0.9146 - val_loss: 0.6857 - val_accuracy: 0.7219

Epoch 00040: val_loss did not improve from 0.58819
Epoch 41/500
37/37 [==============================] - 0s 13ms/step - loss: 0.3005 - accuracy: 0.9161 - val_loss: 0.6966 - val_accuracy: 0.7225

Epoch 00041: val_loss did not improve from 0.58819
Epoch 42/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2960 - accuracy: 0.9165 - val_loss: 0.6935 - val_accuracy: 0.7309

Epoch 00042: val_loss did not improve from 0.58819
Epoch 43/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2884 - accuracy: 0.9195 - val_loss: 0.7048 - val_accuracy: 0.7225

Epoch 00043: val_loss did not improve from 0.58819
Epoch 44/500
37/37 [==================

In [12]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/4068_notallfit-15-0.5882.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7508090614886731, seed_num = 0
정확도 :0.772168284789644, seed_num = 1
정확도 :0.7708737864077669, seed_num = 2
정확도 :0.7844660194174757, seed_num = 3
정확도 :0.7579288025889968, seed_num = 4
정확도 :0.7792880258899676, seed_num = 5
정확도 :0.7656957928802589, seed_num = 6
정확도 :0.7695792880258899, seed_num = 7
정확도 :0.7728155339805826, seed_num = 8
정확도 :0.7611650485436893, seed_num = 9
정확도 :0.7702265372168284, seed_num = 10
정확도 :0.7773462783171521, seed_num = 11
정확도 :0.7637540453074434, seed_num = 12
정확도 :0.7624595469255664, seed_num = 13
정확도 :0.7669902912621359, seed_num = 14
정확도 :0.7792880258899676, seed_num = 15
정확도 :0.7708737864077669, seed_num = 16
정확도 :0.7702265372168284, seed_num = 17
정확도 :0.7682847896440129, seed_num = 18
정확도 :0.7747572815533981, seed_num = 19
정확도 :0.7669902912621359, seed_num = 20
정확도 :0.7611650485436893, seed_num = 21
정확도 :0.7877022653721683, seed_num = 22
정확도 :0.772168284789644, seed_num = 23
정확도 :0.7501618122977346, seed_num = 24
정확도 :0.7682847896440129, seed_num = 2

### (77.8) epoch = 500

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random

with tf.device('/device:GPU:0'):

    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

    # ---------------------
    seed_num = 42
    # ---------------------
    tf.random.set_seed(seed_num)


    def cudnn_lstm_model():
        model = Sequential()
        model.add(CuDNNLSTM(128, input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
        model.add(Activation('hard_sigmoid'))
        model.add(CuDNNLSTM(units=64, return_sequences=True))
        model.add(Activation('hard_sigmoid'))
        model.add(Dropout(0.2))
        model.add(CuDNNLSTM(units=64, return_sequences=True))
        model.add(Activation('hard_sigmoid'))
        model.add(CuDNNLSTM(units=32, return_sequences=False))
        model.add(Activation('hard_sigmoid'))
        model.add(Dropout(0.2))
        model.add(Dense(units=1, activation='sigmoid'))

        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    

        return model

    model = cudnn_lstm_model()


    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'not_allfit_noearlystop-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 3s 26ms/step - loss: 0.6849 - accuracy: 0.5887 - val_loss: 0.6655 - val_accuracy: 0.6177

Epoch 00001: val_loss improved from inf to 0.66551, saving model to ./model/CuDNNLSTM/not_allfit_noearlystop-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6810 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00002: val_loss improved from 0.66551 to 0.66520, saving model to ./model/CuDNNLSTM/not_allfit_noearlystop-02-0.6652.hdf5
Epoch 3/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6814 - accuracy: 0.5951 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00003: val_loss did not improve from 0.66520
Epoch 4/500
37/37 [==============================] - 0s 11ms/step - loss: 0.6772 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00004: val_loss improved from 0.66520 to 0.66517, saving model to ./model/CuDNNLSTM/not_allfit_noearlystop-04-0.6652.hd


Epoch 00037: val_loss did not improve from 0.57768
Epoch 38/500
37/37 [==============================] - 0s 13ms/step - loss: 0.3145 - accuracy: 0.9055 - val_loss: 0.6869 - val_accuracy: 0.7063

Epoch 00038: val_loss did not improve from 0.57768
Epoch 39/500
37/37 [==============================] - 0s 11ms/step - loss: 0.3167 - accuracy: 0.9079 - val_loss: 0.6729 - val_accuracy: 0.7283

Epoch 00039: val_loss did not improve from 0.57768
Epoch 40/500
37/37 [==============================] - 0s 13ms/step - loss: 0.3066 - accuracy: 0.9139 - val_loss: 0.6865 - val_accuracy: 0.7199

Epoch 00040: val_loss did not improve from 0.57768
Epoch 41/500
37/37 [==============================] - 0s 11ms/step - loss: 0.3058 - accuracy: 0.9135 - val_loss: 0.6960 - val_accuracy: 0.7212

Epoch 00041: val_loss did not improve from 0.57768
Epoch 42/500
37/37 [==============================] - 0s 13ms/step - loss: 0.3009 - accuracy: 0.9163 - val_loss: 0.7036 - val_accuracy: 0.7122

Epoch 00042: val_loss di

37/37 [==============================] - 0s 13ms/step - loss: 0.2171 - accuracy: 0.9484 - val_loss: 0.8386 - val_accuracy: 0.7128

Epoch 00080: val_loss did not improve from 0.57768
Epoch 81/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2219 - accuracy: 0.9480 - val_loss: 0.8415 - val_accuracy: 0.7102

Epoch 00081: val_loss did not improve from 0.57768
Epoch 82/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2163 - accuracy: 0.9493 - val_loss: 0.8476 - val_accuracy: 0.7089

Epoch 00082: val_loss did not improve from 0.57768
Epoch 83/500
37/37 [==============================] - 0s 13ms/step - loss: 0.2132 - accuracy: 0.9502 - val_loss: 0.8518 - val_accuracy: 0.7044

Epoch 00083: val_loss did not improve from 0.57768
Epoch 84/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2170 - accuracy: 0.9504 - val_loss: 0.8527 - val_accuracy: 0.7122

Epoch 00084: val_loss did not improve from 0.57768
Epoch 85/500
37/37 [==================

37/37 [==============================] - 0s 13ms/step - loss: 0.2269 - accuracy: 0.9446 - val_loss: 0.8931 - val_accuracy: 0.7038

Epoch 00122: val_loss did not improve from 0.57768
Epoch 123/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2209 - accuracy: 0.9467 - val_loss: 0.8889 - val_accuracy: 0.7122

Epoch 00123: val_loss did not improve from 0.57768
Epoch 124/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2187 - accuracy: 0.9478 - val_loss: 0.8618 - val_accuracy: 0.7212

Epoch 00124: val_loss did not improve from 0.57768
Epoch 125/500
37/37 [==============================] - 0s 12ms/step - loss: 0.2160 - accuracy: 0.9478 - val_loss: 0.9184 - val_accuracy: 0.6973

Epoch 00125: val_loss did not improve from 0.57768
Epoch 126/500
37/37 [==============================] - 0s 11ms/step - loss: 0.2115 - accuracy: 0.9500 - val_loss: 0.8819 - val_accuracy: 0.7083

Epoch 00126: val_loss did not improve from 0.57768
Epoch 127/500
37/37 [=============

37/37 [==============================] - 0s 13ms/step - loss: 0.1522 - accuracy: 0.9668 - val_loss: 0.9697 - val_accuracy: 0.7057

Epoch 00164: val_loss did not improve from 0.57768
Epoch 165/500
37/37 [==============================] - 0s 11ms/step - loss: 0.1512 - accuracy: 0.9668 - val_loss: 0.9736 - val_accuracy: 0.7083

Epoch 00165: val_loss did not improve from 0.57768
Epoch 166/500
37/37 [==============================] - 0s 12ms/step - loss: 0.1438 - accuracy: 0.9692 - val_loss: 0.9730 - val_accuracy: 0.7089

Epoch 00166: val_loss did not improve from 0.57768
Epoch 167/500
37/37 [==============================] - 0s 11ms/step - loss: 0.1388 - accuracy: 0.9696 - val_loss: 0.9761 - val_accuracy: 0.7141

Epoch 00167: val_loss did not improve from 0.57768
Epoch 168/500
37/37 [==============================] - 0s 13ms/step - loss: 0.1367 - accuracy: 0.9715 - val_loss: 0.9852 - val_accuracy: 0.7076

Epoch 00168: val_loss did not improve from 0.57768
Epoch 169/500
37/37 [=============

...

37/37 [==============================] - 0s 13ms/step - loss: 0.1040 - accuracy: 0.9793 - val_loss: 1.3507 - val_accuracy: 0.7219

Epoch 00332: val_loss did not improve from 0.57768
Epoch 333/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0770 - accuracy: 0.9845 - val_loss: 1.4899 - val_accuracy: 0.6979

Epoch 00333: val_loss did not improve from 0.57768
Epoch 334/500
37/37 [==============================] - 0s 12ms/step - loss: 0.0810 - accuracy: 0.9862 - val_loss: 1.3761 - val_accuracy: 0.7063

Epoch 00334: val_loss did not improve from 0.57768
Epoch 335/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0756 - accuracy: 0.9860 - val_loss: 1.4145 - val_accuracy: 0.7063

Epoch 00335: val_loss did not improve from 0.57768
Epoch 336/500
37/37 [==============================] - 0s 13ms/step - loss: 0.0675 - accuracy: 0.9879 - val_loss: 1.4510 - val_accuracy: 0.7096

Epoch 00336: val_loss did not improve from 0.57768
Epoch 337/500
37/37 [=============

37/37 [==============================] - 0s 13ms/step - loss: 0.0497 - accuracy: 0.9925 - val_loss: 1.5619 - val_accuracy: 0.6999

Epoch 00374: val_loss did not improve from 0.57768
Epoch 375/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0514 - accuracy: 0.9922 - val_loss: 1.5696 - val_accuracy: 0.7018

Epoch 00375: val_loss did not improve from 0.57768
Epoch 376/500
37/37 [==============================] - 1s 14ms/step - loss: 0.0534 - accuracy: 0.9920 - val_loss: 1.5645 - val_accuracy: 0.7031

Epoch 00376: val_loss did not improve from 0.57768
Epoch 377/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0526 - accuracy: 0.9931 - val_loss: 1.5645 - val_accuracy: 0.7025

Epoch 00377: val_loss did not improve from 0.57768
Epoch 378/500
37/37 [==============================] - 0s 13ms/step - loss: 0.0487 - accuracy: 0.9929 - val_loss: 1.5624 - val_accuracy: 0.7044

Epoch 00378: val_loss did not improve from 0.57768
Epoch 379/500
37/37 [=============

37/37 [==============================] - 0s 13ms/step - loss: 0.0665 - accuracy: 0.9888 - val_loss: 1.4410 - val_accuracy: 0.7083

Epoch 00416: val_loss did not improve from 0.57768
Epoch 417/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0609 - accuracy: 0.9916 - val_loss: 1.5366 - val_accuracy: 0.6986

Epoch 00417: val_loss did not improve from 0.57768
Epoch 418/500
37/37 [==============================] - 0s 13ms/step - loss: 0.0526 - accuracy: 0.9920 - val_loss: 1.5069 - val_accuracy: 0.7038

Epoch 00418: val_loss did not improve from 0.57768
Epoch 419/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0502 - accuracy: 0.9922 - val_loss: 1.4986 - val_accuracy: 0.7154

Epoch 00419: val_loss did not improve from 0.57768
Epoch 420/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0492 - accuracy: 0.9931 - val_loss: 1.5191 - val_accuracy: 0.7122

Epoch 00420: val_loss did not improve from 0.57768
Epoch 421/500
37/37 [=============

37/37 [==============================] - 0s 12ms/step - loss: 0.0340 - accuracy: 0.9953 - val_loss: 1.6633 - val_accuracy: 0.7076

Epoch 00458: val_loss did not improve from 0.57768
Epoch 459/500
37/37 [==============================] - 0s 12ms/step - loss: 0.0372 - accuracy: 0.9953 - val_loss: 1.6680 - val_accuracy: 0.7070

Epoch 00459: val_loss did not improve from 0.57768
Epoch 460/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0379 - accuracy: 0.9946 - val_loss: 1.6500 - val_accuracy: 0.7109

Epoch 00460: val_loss did not improve from 0.57768
Epoch 461/500
37/37 [==============================] - 0s 13ms/step - loss: 0.0386 - accuracy: 0.9940 - val_loss: 1.6528 - val_accuracy: 0.7115

Epoch 00461: val_loss did not improve from 0.57768
Epoch 462/500
37/37 [==============================] - 0s 11ms/step - loss: 0.0372 - accuracy: 0.9942 - val_loss: 1.6543 - val_accuracy: 0.7122

Epoch 00462: val_loss did not improve from 0.57768
Epoch 463/500
37/37 [=============

37/37 [==============================] - 0s 9ms/step - loss: 0.0427 - accuracy: 0.9940 - val_loss: 1.6084 - val_accuracy: 0.7102

Epoch 00500: val_loss did not improve from 0.57768


In [4]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/not_allfit_noearlystop-19-0.5777.hdf5') 

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7611650485436893, seed_num = 0
정확도 :0.7922330097087379, seed_num = 1
정확도 :0.7779935275080906, seed_num = 2
정확도 :0.7870550161812297, seed_num = 3
정확도 :0.7805825242718447, seed_num = 4
정확도 :0.7896440129449838, seed_num = 5
정확도 :0.7754045307443366, seed_num = 6
정확도 :0.7818770226537217, seed_num = 7
정확도 :0.7877022653721683, seed_num = 8
정확도 :0.7805825242718447, seed_num = 9
정확도 :0.7935275080906149, seed_num = 10
정확도 :0.7766990291262136, seed_num = 11
정확도 :0.7773462783171521, seed_num = 12
정확도 :0.7902912621359224, seed_num = 13
정확도 :0.7786407766990291, seed_num = 14
정확도 :0.7831715210355987, seed_num = 15
정확도 :0.7779935275080906, seed_num = 16
정확도 :0.7689320388349514, seed_num = 17
정확도 :0.7689320388349514, seed_num = 18
정확도 :0.7851132686084142, seed_num = 19
정확도 :0.7766990291262136, seed_num = 20
정확도 :0.7715210355987056, seed_num = 21
정확도 :0.7818770226537217, seed_num = 22
정확도 :0.7844660194174757, seed_num = 23
정확도 :0.7656957928802589, seed_num = 24
정확도 :0.7870550161812297, seed_num =

### (77.8) epoch = 1000

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2

import random

with tf.device('/device:GPU:0'):

    # ----------------------
    seed_num = 42
    # ----------------------
    random.seed(seed_num)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

    # ---------------------
    seed_num = 42
    # ---------------------
    tf.random.set_seed(seed_num)


    def cudnn_lstm_model():
        model = Sequential()
        model.add(CuDNNLSTM(128, input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
        model.add(Activation('hard_sigmoid'))
        model.add(CuDNNLSTM(units=64, return_sequences=True))
        model.add(Activation('hard_sigmoid'))
        model.add(Dropout(0.2))
        model.add(CuDNNLSTM(units=64, return_sequences=True))
        model.add(Activation('hard_sigmoid'))
        model.add(CuDNNLSTM(units=32, return_sequences=False))
        model.add(Activation('hard_sigmoid'))
        model.add(Dropout(0.2))
        model.add(Dense(units=1, activation='sigmoid'))

        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    

        return model

    model = cudnn_lstm_model()


    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    MODEL_SAVE_FOLDER_PATH = './model/CuDNNLSTM/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + 'epoch1000_notallfit-{epoch:02d}-{val_loss:.4f}.hdf5'

    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                    verbose=1, save_best_only=True)

    model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=1000,  callbacks=[cb_checkpoint], shuffle=False)

Epoch 1/1000
37/37 [==============================] - 2s 32ms/step - loss: 0.6849 - accuracy: 0.5887 - val_loss: 0.6655 - val_accuracy: 0.6177

Epoch 00001: val_loss improved from inf to 0.66551, saving model to ./model/CuDNNLSTM/epoch1000_notallfit-01-0.6655.hdf5
Epoch 2/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.6810 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00002: val_loss improved from 0.66551 to 0.66520, saving model to ./model/CuDNNLSTM/epoch1000_notallfit-02-0.6652.hdf5
Epoch 3/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.6814 - accuracy: 0.5951 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00003: val_loss did not improve from 0.66520
Epoch 4/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.6772 - accuracy: 0.5988 - val_loss: 0.6652 - val_accuracy: 0.6177

Epoch 00004: val_loss improved from 0.66520 to 0.66517, saving model to ./model/CuDNNLSTM/epoch1000_notallfit-04-0.6652.hdf5
Ep


Epoch 00037: val_loss did not improve from 0.57768
Epoch 38/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.3145 - accuracy: 0.9055 - val_loss: 0.6869 - val_accuracy: 0.7063

Epoch 00038: val_loss did not improve from 0.57768
Epoch 39/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.3167 - accuracy: 0.9079 - val_loss: 0.6729 - val_accuracy: 0.7283

Epoch 00039: val_loss did not improve from 0.57768
Epoch 40/1000
37/37 [==============================] - 1s 14ms/step - loss: 0.3066 - accuracy: 0.9139 - val_loss: 0.6865 - val_accuracy: 0.7199

Epoch 00040: val_loss did not improve from 0.57768
Epoch 41/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.3058 - accuracy: 0.9135 - val_loss: 0.6960 - val_accuracy: 0.7212

Epoch 00041: val_loss did not improve from 0.57768
Epoch 42/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.3009 - accuracy: 0.9163 - val_loss: 0.7036 - val_accuracy: 0.7122

Epoch 00042: val_lo


Epoch 00079: val_loss did not improve from 0.57768
Epoch 80/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.2171 - accuracy: 0.9484 - val_loss: 0.8386 - val_accuracy: 0.7128

Epoch 00080: val_loss did not improve from 0.57768
Epoch 81/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.2219 - accuracy: 0.9480 - val_loss: 0.8415 - val_accuracy: 0.7102

Epoch 00081: val_loss did not improve from 0.57768
Epoch 82/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.2163 - accuracy: 0.9493 - val_loss: 0.8476 - val_accuracy: 0.7089

Epoch 00082: val_loss did not improve from 0.57768
Epoch 83/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.2132 - accuracy: 0.9502 - val_loss: 0.8518 - val_accuracy: 0.7044

Epoch 00083: val_loss did not improve from 0.57768
Epoch 84/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.2170 - accuracy: 0.9504 - val_loss: 0.8527 - val_accuracy: 0.7122

Epoch 00084: val_lo

37/37 [==============================] - 0s 13ms/step - loss: 0.2125 - accuracy: 0.9487 - val_loss: 0.8619 - val_accuracy: 0.7186

Epoch 00121: val_loss did not improve from 0.57768
Epoch 122/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.2269 - accuracy: 0.9446 - val_loss: 0.8931 - val_accuracy: 0.7038

Epoch 00122: val_loss did not improve from 0.57768
Epoch 123/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.2209 - accuracy: 0.9467 - val_loss: 0.8889 - val_accuracy: 0.7122

Epoch 00123: val_loss did not improve from 0.57768
Epoch 124/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.2187 - accuracy: 0.9478 - val_loss: 0.8618 - val_accuracy: 0.7212

Epoch 00124: val_loss did not improve from 0.57768
Epoch 125/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.2160 - accuracy: 0.9478 - val_loss: 0.9184 - val_accuracy: 0.6973

Epoch 00125: val_loss did not improve from 0.57768
Epoch 126/1000
37/37 [========


Epoch 00162: val_loss did not improve from 0.57768
Epoch 163/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.1532 - accuracy: 0.9642 - val_loss: 0.9788 - val_accuracy: 0.7076

Epoch 00163: val_loss did not improve from 0.57768
Epoch 164/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.1522 - accuracy: 0.9668 - val_loss: 0.9697 - val_accuracy: 0.7057

Epoch 00164: val_loss did not improve from 0.57768
Epoch 165/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.1512 - accuracy: 0.9668 - val_loss: 0.9736 - val_accuracy: 0.7083

Epoch 00165: val_loss did not improve from 0.57768
Epoch 166/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.1438 - accuracy: 0.9692 - val_loss: 0.9730 - val_accuracy: 0.7089

Epoch 00166: val_loss did not improve from 0.57768
Epoch 167/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.1388 - accuracy: 0.9696 - val_loss: 0.9761 - val_accuracy: 0.7141

Epoch 00167: v

...


Epoch 00826: val_loss did not improve from 0.57768
Epoch 827/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0184 - accuracy: 0.9976 - val_loss: 2.1610 - val_accuracy: 0.7096

Epoch 00827: val_loss did not improve from 0.57768
Epoch 828/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0154 - accuracy: 0.9976 - val_loss: 2.1690 - val_accuracy: 0.7102

Epoch 00828: val_loss did not improve from 0.57768
Epoch 829/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0147 - accuracy: 0.9978 - val_loss: 2.1842 - val_accuracy: 0.7076

Epoch 00829: val_loss did not improve from 0.57768
Epoch 830/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0173 - accuracy: 0.9978 - val_loss: 2.1795 - val_accuracy: 0.7076

Epoch 00830: val_loss did not improve from 0.57768
Epoch 831/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0152 - accuracy: 0.9981 - val_loss: 2.1856 - val_accuracy: 0.7025

Epoch 00831: v

37/37 [==============================] - 0s 11ms/step - loss: 0.0195 - accuracy: 0.9976 - val_loss: 2.1399 - val_accuracy: 0.7050

Epoch 00868: val_loss did not improve from 0.57768
Epoch 869/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0194 - accuracy: 0.9983 - val_loss: 2.1388 - val_accuracy: 0.7038

Epoch 00869: val_loss did not improve from 0.57768
Epoch 870/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0202 - accuracy: 0.9976 - val_loss: 2.1485 - val_accuracy: 0.7031

Epoch 00870: val_loss did not improve from 0.57768
Epoch 871/1000
37/37 [==============================] - 1s 14ms/step - loss: 0.0202 - accuracy: 0.9974 - val_loss: 2.1620 - val_accuracy: 0.7044

Epoch 00871: val_loss did not improve from 0.57768
Epoch 872/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0167 - accuracy: 0.9981 - val_loss: 2.1728 - val_accuracy: 0.7018

Epoch 00872: val_loss did not improve from 0.57768
Epoch 873/1000
37/37 [========


Epoch 00909: val_loss did not improve from 0.57768
Epoch 910/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0108 - accuracy: 0.9985 - val_loss: 2.2694 - val_accuracy: 0.6889

Epoch 00910: val_loss did not improve from 0.57768
Epoch 911/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0111 - accuracy: 0.9987 - val_loss: 2.2786 - val_accuracy: 0.6889

Epoch 00911: val_loss did not improve from 0.57768
Epoch 912/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0093 - accuracy: 0.9987 - val_loss: 2.2780 - val_accuracy: 0.6882

Epoch 00912: val_loss did not improve from 0.57768
Epoch 913/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0121 - accuracy: 0.9983 - val_loss: 2.2821 - val_accuracy: 0.6831

Epoch 00913: val_loss did not improve from 0.57768
Epoch 914/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0101 - accuracy: 0.9983 - val_loss: 2.2893 - val_accuracy: 0.6882

Epoch 00914: v

37/37 [==============================] - 0s 12ms/step - loss: 0.0147 - accuracy: 0.9983 - val_loss: 2.4117 - val_accuracy: 0.6895

Epoch 00951: val_loss did not improve from 0.57768
Epoch 952/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0122 - accuracy: 0.9987 - val_loss: 2.3907 - val_accuracy: 0.6947

Epoch 00952: val_loss did not improve from 0.57768
Epoch 953/1000
37/37 [==============================] - 0s 14ms/step - loss: 0.0117 - accuracy: 0.9983 - val_loss: 2.3695 - val_accuracy: 0.7012

Epoch 00953: val_loss did not improve from 0.57768
Epoch 954/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0103 - accuracy: 0.9981 - val_loss: 2.4539 - val_accuracy: 0.6843

Epoch 00954: val_loss did not improve from 0.57768
Epoch 955/1000
37/37 [==============================] - 0s 14ms/step - loss: 0.0372 - accuracy: 0.9931 - val_loss: 1.9686 - val_accuracy: 0.7050

Epoch 00955: val_loss did not improve from 0.57768
Epoch 956/1000
37/37 [========


Epoch 00992: val_loss did not improve from 0.57768
Epoch 993/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0062 - accuracy: 0.9991 - val_loss: 2.5577 - val_accuracy: 0.6947

Epoch 00993: val_loss did not improve from 0.57768
Epoch 994/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0054 - accuracy: 0.9994 - val_loss: 2.5672 - val_accuracy: 0.6940

Epoch 00994: val_loss did not improve from 0.57768
Epoch 995/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.0081 - accuracy: 0.9983 - val_loss: 2.5908 - val_accuracy: 0.6882

Epoch 00995: val_loss did not improve from 0.57768
Epoch 996/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.0088 - accuracy: 0.9983 - val_loss: 2.5069 - val_accuracy: 0.6947

Epoch 00996: val_loss did not improve from 0.57768
Epoch 997/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.0072 - accuracy: 0.9989 - val_loss: 2.5945 - val_accuracy: 0.6850

Epoch 00997: v

In [10]:
# ----------------------------
seed_num = 42
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/CuDNNLSTM/epoch1000_notallfit-19-0.5777.hdf5') 

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7611650485436893, seed_num = 0
정확도 :0.7922330097087379, seed_num = 1
정확도 :0.7779935275080906, seed_num = 2
정확도 :0.7870550161812297, seed_num = 3
정확도 :0.7805825242718447, seed_num = 4
정확도 :0.7896440129449838, seed_num = 5
정확도 :0.7754045307443366, seed_num = 6
정확도 :0.7818770226537217, seed_num = 7
정확도 :0.7877022653721683, seed_num = 8
정확도 :0.7805825242718447, seed_num = 9
정확도 :0.7935275080906149, seed_num = 10
정확도 :0.7766990291262136, seed_num = 11
정확도 :0.7773462783171521, seed_num = 12
정확도 :0.7902912621359224, seed_num = 13
정확도 :0.7786407766990291, seed_num = 14
정확도 :0.7831715210355987, seed_num = 15
정확도 :0.7779935275080906, seed_num = 16
정확도 :0.7689320388349514, seed_num = 17
정확도 :0.7689320388349514, seed_num = 18
정확도 :0.7851132686084142, seed_num = 19
정확도 :0.7766990291262136, seed_num = 20
정확도 :0.7715210355987056, seed_num = 21
정확도 :0.7818770226537217, seed_num = 22
정확도 :0.7844660194174757, seed_num = 23
정확도 :0.7656957928802589, seed_num = 24
정확도 :0.7870550161812297, seed_num =